In [3]:
import pickle
import copy
import json
import numpy as np
import random
import osmnx as ox
import networkx as nx
from osmnx.core import osm_net_download

### 导入地图数据及其他基础数据

In [4]:
G = pickle.load(open("./data/beijing_osmnx_networkx.obj","rb"))
trip = np.load('./data/real_od_9am.npy')
nodes_mapping = json.load(open("./data/net_dictionary.rs"))
nodes_mapping_inverse = dict([(v,int(k)) for (k,v) in nodes_mapping.items()])
cityflow_road = json.load(open("data/beijing_cityflow_L2_fixed.json"))

### 读取OD数据 （可随机采样生成）

In [5]:
node = {}
for index,data in enumerate(G.nodes(data=True)):
    node[data[0]] = index+1
nodes = {k:v for v,k in node.items()}

cnt = 0
for line in open('./data/9.00am-9.10am.txt','r'):
    a = line
    u = int(a[a.find(':')+1:a.find('-')])
    v = int(a[a.find('>')+1:a.find(' ')])
    flo = float(a[a.find('time:')+5:a.find('\n')])
    cnt = cnt+1
    
    for j in range(len(G.get_edge_data(nodes[u+1],nodes[v+1]))):
        try:
            if G.get_edge_data(nodes[u+1],nodes[v+1])[j]['weight']:
                continue
        except KeyError:
            break
            
    G.get_edge_data(nodes[u+1],nodes[v+1])[j]['weight'] = flo

### 处理道路信息，为生成轨迹做准备

In [6]:
# 统计路口id的道路连接情况
roads_for_nodes = {}
for node in cityflow_road["intersections"]:
    roads_for_nodes[node["id"]] = {"enter":[],"leave":[]}
    for link in node["roadLinks"]:
        if link["startRoad"] not in roads_for_nodes[node["id"]]["enter"]:
            roads_for_nodes[node["id"]]["enter"].append(link["startRoad"])
        if link["endRoad"] not in roads_for_nodes[node["id"]]["leave"]:
            roads_for_nodes[node["id"]]["leave"].append(link["endRoad"])

# 统计路口id的道路lane
road_lanes = {}
for road in cityflow_road["roads"]:
    road_lanes[road["id"]] = len(road["lanes"])
    
# 给定有序的路口id，给出其连接路段id
path_for_nodes = {}
for node_src in roads_for_nodes:
    for node_dst in roads_for_nodes:
        if node_src==node_dst:
            continue
        path_index = str(node_src) + "->" + str(node_dst)
        leave_road = set(roads_for_nodes[str(node_src)]["leave"])
        enter_road = set(roads_for_nodes[str(node_dst)]["enter"])
        if len(leave_road.intersection(enter_road))==0:
            continue
        else:
            path_for_nodes[path_index] = list(leave_road.intersection(enter_road))[0]

# virtual nodes
virtual_nodes = {"nolink":[],"virtual":[], "merge":[]}
for node in cityflow_road["intersections"]:
    if len(node["roadLinks"])==0:
        virtual_nodes["nolink"].append(node["id"])
    if str(node["virtual"])=="True":
        virtual_nodes["virtual"].append(node["id"])
virtual_nodes["merge"] = list(set(virtual_nodes["nolink"])|set(virtual_nodes["virtual"]))

### 车辆信息模板

In [7]:
vehicle_head = {
   "length": 5.0,
   "width": 2.0,
   "maxPosAcc": 2.0,
   "maxNegAcc": 4.5,
   "usualPosAcc": 2.0,
   "usualNegAcc": 4.5,
   "minGap": 2.5,
   "maxSpeed": 11.111,
   "headwayTime": 2
  }

### 基于道路信息和OD信息，生成轨迹数据

In [8]:
random.seed(1)
vehicle_flow = []
change, error = 0, 0
for src_node in list(G.nodes()):
    # 处理虚拟节点
    if str(src_node) in virtual_nodes["merge"]:
        continue
    for dst_node in list(G.nodes()):
        if str(dst_node) in virtual_nodes["merge"]:
            continue
        
        # 处理flow为0的点
        flow_value = trip[0, nodes_mapping_inverse[str(src_node)], nodes_mapping_inverse[str(dst_node)]]
        if flow_value == 0:
            continue
        
        node_list = nx.shortest_path(G, source=src_node, target=dst_node)
        edge_nodes = list(zip(node_list[:-1], node_list[1:]))
        
        path_list = []
        flag = 0 # 进一步处理node
        for eu, ev in edge_nodes:
            if str(eu)+"->"+str(ev) not in path_for_nodes or str(eu) in virtual_nodes["merge"] or str(ev) in virtual_nodes["merge"]:
                flag = 1
                break
            path_list.append(path_for_nodes[str(eu)+"->"+str(ev)])
        if flag:
            error += 1
            print("error caused by virtual intersection:", src_node, dst_node, node_list,str(eu)+"->"+str(ev))
            continue

        for i in range(max(int(flow_value/2),1)):
            random_time = random.randint(0,600) # random generate time in 600s
#             threshold = 2 if random.randint(1,10)<5 else 0 
#             if random.randint(1,10)<4:
#                 random_time = random.randint(0,1800)
#             else:
#                 random_time = random.randint(300,600)
            vehicle_flow_unit = {"vehicle":vehicle_head,
                                 "route":path_list,
                                 "interval": 1.0,
                                 "startTime": random_time,
                                 "endTime": random_time
                                }
            vehicle_flow.append(vehicle_flow_unit)
vehicle_flow_order = sorted(vehicle_flow, key=lambda x: x["startTime"])
json.dump(vehicle_flow_order, open("./data/beijing_vehicle_flow_L2_300.json","w"),indent=2)

error caused by virtual intersection: 528909202 1982049864 [528909202, 4580060672, 528911157, 2108372983, 1365113263, 2285990048, 2285990050, 265710690, 1982049864] 528911157->2108372983
error caused by virtual intersection: 528911157 1253535147 [528911157, 2108372983, 1365113263, 2285990048, 1365113238, 290598252, 656086849, 1253535147] 528911157->2108372983
error caused by virtual intersection: 1305972690 1449318308 [1305972690, 35552589, 76456821, 1942597351, 76456822, 1842188224, 1736610693, 35552324, 32899017, 1736610643, 1114097887, 1114088466, 1114088475, 1114088468, 1819087374, 1819087334, 1819087329, 1977226294, 100651857, 266110505, 100653372, 1300406443, 31018238, 336380069, 266112314, 1362315796, 1861654993, 1362315802, 1449318308] 100653372->1300406443
error caused by virtual intersection: 734170185 1598459387 [734170185, 734169511, 77746334, 2480776634, 1599489635, 77746332, 1549077228, 322121114, 1231628428, 322121120, 1410714249, 734418814, 1598453939, 1598459387] 32212